<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173_Fall2025/blob/main/F25_Class_02_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 2: Neural Networks with Tensorflow and Keras**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 2 Material

* Part 2.1: Introduction to Neural Networks with Tensorflow and Keras
* Part 2.2: Encoding Feature Vectors
* Part 2.3: Early Stopping and Dropout to Prevent Overfitting
* **Part 2.4: Saving and Loading a Keras Neural Network**

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [1]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

Mounted at /content/drive
Note: Using Google CoLab
david.senseman@gmail.com


Make sure your GMAIL address is included as the last line in the output above.

### Define functions

The cell below creates the function(s) needed for this lesson.

In [2]:
# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Simple function to change column name in a dataframe
def rename_col_by_index(dataframe, index_mapping):
    dataframe.columns = [index_mapping.get(i, col) for i, col in enumerate(dataframe.columns)]
    return dataframe

# **Saving and Loading a Keras Neural Network**

Complex neural networks will take a _long_ time to fit/train.  It is helpful to be able to save a trained neural network so that you can reload it and using it again.  Again, a reloaded neural network will **not** require retraining.  

Keras provides the following two formats for saving neural networks:

* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **Keras** - Stores the complete neural network (with weights) in the native Keras format.

Usually, you will want to save in native Keras format.

## **Early Stopping**

We will now see an example of classification training with early stopping. We will train a neural network until the error no longer improves on the validation set.

### Example 1: Build and Train a Classification Neural Network

The code in Example 1 builds and trains a neural network called `orModel` that can classify the `Quality` of an orange based on its physical and chemical characteristics.

The code in the cell below reads the Orange Quality dataset from the course HTTP server and creates a DataFrame called `orDF` (i.e. "orange" DataFrame).

In order to create a feature vector, the 3 non-numeric columns in the dataset: `Color`, `Variety` and `Blemished` must be pre-processed. Mapping strings to integers is used to take care of the column `Color` while One-Hot Encoding is used to take care the column `Variety`. To take care of the column `Blemish`, it will simply excluded (dropped) from the column list when generating the X-values.

While most of the numerical values are small integers, the weight values are an order of magnitude larger, so they are standardized to their z-scores. This makes weight values closer in magnitude to the other numerical values and significantly improved the accuracy of the model. After the weights are stardardized, X-values are generated and stored in a Numpy array called `orX`.

For this assigment we won't bother to split the data into Training/Validation sets, nor will we bother to shuffle the data.

Since we are building a classification neural network, we will need to One-Hot Encode the column `Quality` which contains the Y-values. It should be noted that this column is already numeric, so we are **not** using One-Hot Encoding to replace string values with integer. Rather, One-Hot Encoding the Y-values is necessary to give them the **_correct format_** for the neural network.

Again, because we want `orModel` to act as a _classifier_, not a "regressor", we will use the `softmax` activation function in the output layer.

Finally, we will compile the model using `categorical_crossentropy` as the loss function instead of using `mean_squared_error`.   


In [3]:
# Example 1: Build and Train Classification Model

import pandas as pd
import numpy as np
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
import numpy as np

# Read dataset and create DataFrame --------------------------------
orDF = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/orange_quality.csv",
    na_values=['NA', '?'])


# Create feature vector ---------------------------------------------

# Map str to int
mapping = {'Orange':0,'Deep Orange':1,
           'Light Orange':2,'Orange-Red':3,
           'Yellow-Orange':4}
orDF['Color'] = orDF['Color'].map(mapping)

# Standardize to z-scores
orDF['Weight (g)'] = zscore(orDF['Weight (g)'])

# Generate X-values
orX = orDF[['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)',
       'Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)',
        'Color']].values
orX = np.asarray(orX).astype('float32')

# Generate Y-values
dummies = pd.get_dummies(orDF['Quality (1-5)'], dtype=int) # Classification
orY = dummies.values
orY = np.asarray(orY).astype('float32')

# Build neural network-----------------------------------------
orModel = Sequential()
orModel.add(Input(shape=(orX.shape[1],)))  # Input
orModel.add(Dense(50, activation='relu')) # Hidden 1
orModel.add(Dense(25, activation='relu')) # Hidden 2
orModel.add(Dense(orY.shape[1],activation='softmax')) # Output
orModel.compile(loss='categorical_crossentropy', optimizer='adam')

# Train model
orModel.fit(orX,orY,verbose=2,epochs=100)

Epoch 1/100
8/8 - 4s - 508ms/step - loss: 3.5602
Epoch 2/100
8/8 - 0s - 7ms/step - loss: 2.7233
Epoch 3/100
8/8 - 0s - 6ms/step - loss: 2.1594
Epoch 4/100
8/8 - 0s - 7ms/step - loss: 1.8612
Epoch 5/100
8/8 - 0s - 7ms/step - loss: 1.7471
Epoch 6/100
8/8 - 0s - 7ms/step - loss: 1.6832
Epoch 7/100
8/8 - 0s - 6ms/step - loss: 1.6344
Epoch 8/100
8/8 - 0s - 7ms/step - loss: 1.6052
Epoch 9/100
8/8 - 0s - 7ms/step - loss: 1.5773
Epoch 10/100
8/8 - 0s - 7ms/step - loss: 1.5609
Epoch 11/100
8/8 - 0s - 6ms/step - loss: 1.5462
Epoch 12/100
8/8 - 0s - 7ms/step - loss: 1.5318
Epoch 13/100
8/8 - 0s - 6ms/step - loss: 1.5140
Epoch 14/100
8/8 - 0s - 7ms/step - loss: 1.5018
Epoch 15/100
8/8 - 0s - 7ms/step - loss: 1.4912
Epoch 16/100
8/8 - 0s - 6ms/step - loss: 1.4780
Epoch 17/100
8/8 - 0s - 7ms/step - loss: 1.4734
Epoch 18/100
8/8 - 0s - 7ms/step - loss: 1.4593
Epoch 19/100
8/8 - 0s - 6ms/step - loss: 1.4503
Epoch 20/100
8/8 - 0s - 6ms/step - loss: 1.4357
Epoch 21/100
8/8 - 0s - 6ms/step - loss: 1.4329

Training should go pretty fast since there are only 241 oranges in the dataset.

If you code is correct, you should see something similar to the following output:

~~~text
Epoch 1/100
8/8 - 2s - 274ms/step - loss: 4.0258
Epoch 2/100
8/8 - 0s - 7ms/step - loss: 3.0507
Epoch 3/100
8/8 - 0s - 8ms/step - loss: 2.3814
Epoch 4/100
8/8 - 0s - 7ms/step - loss: 1.9822
Epoch 5/100
8/8 - 0s - 5ms/step - loss: 1.8397

.......................................

Epoch 95/100
8/8 - 0s - 8ms/step - loss: 1.1751
Epoch 96/100
8/8 - 0s - 7ms/step - loss: 1.1711
Epoch 97/100
8/8 - 0s - 7ms/step - loss: 1.1699
Epoch 98/100
8/8 - 0s - 5ms/step - loss: 1.1656
Epoch 99/100
8/8 - 0s - 7ms/step - loss: 1.1683
Epoch 100/100
8/8 - 0s - 5ms/step - loss: 1.1603
<keras.src.callbacks.history.History at 0x7e7bb6d79060>

~~~

### **Exercise 1: Build and Train a Classification Neural Network**

In the cell below build and train a new classification neural network called `apModel`.

Start by reading the dataset and creating a DataFrame called `apDF` ("apple" DataFrame) using this code chunk:
~~~text
apDF = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/apple_quality.csv",
    na_values=['NA', '?'])
~~~
The goal of your neural network model `apModel` will be to classify the apples in the Apple Quality dataset using the values in the following columns: 'Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Acidity' and 'Ripeness'. Since all of these columns are numeric, there is no need pre-process any of these columns. Moreover, the numerical values all have a similar magnitude so you don't need to standardize any column to their z-scores. Nor do you need to split the data into Training/Validation sets or suffle the data. When you generate your X-values, you should called them `apX`.

Since you are building a classification neural network, you will need to One-Hot Encode the column containing the Y-values, `Quality`. This column is non-numeric, so by One-Hot Encoding it, you will accomplish two things: (1) replace string values with integers and (2) give the Y-values the correct format for the neural network. When you generate your Y-values, you should called them `apY`.

Again, because you want your `apModel` to act as a _classifier_, use the `softmax` activation function in the output layer. You should also compile your model using `categorical_crossentropy` as the loss function.

Finally, train (fit) your model on your X-values (`apX`) and your Y-values (`apY`) for 100 epochs.  

In [4]:
# Insert your code for Exercise 1 here

# Example 1: Build and Train Classification Model

import pandas as pd
import numpy as np
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
import numpy as np

# Read dataset and create DataFrame --------------------------------
apDF = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/apple_quality.csv",
    na_values=['NA', '?'])


# Create feature vector ---------------------------------------------

#  'Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Acidity' and 'Ripeness'.

# Generate list of columns for x
apX_columns = apDF.columns.drop('Quality')  # `Quality` is y-value

# Generate x-values as numpy array
apX = apDF[apX_columns].values
# Convert x-values to float 32
apX = np.asarray(apX).astype('float32')

# Generate Y-values
dummies = pd.get_dummies(apDF['Quality'], dtype=int) # Classification
apY = dummies.values
apY = np.asarray(apY).astype('float32')

# Build neural network-----------------------------------------
apModel = Sequential()
apModel.add(Input(shape=(apX.shape[1],)))  # Input
apModel.add(Dense(50, activation='relu')) # Hidden 1
apModel.add(Dense(25, activation='relu')) # Hidden 2
apModel.add(Dense(apY.shape[1],activation='softmax')) # Output
apModel.compile(loss='categorical_crossentropy', optimizer='adam')

# Train model
apModel.fit(apX,apY,verbose=2,epochs=100)

Epoch 1/100
125/125 - 2s - 18ms/step - loss: 7.3257
Epoch 2/100
125/125 - 0s - 2ms/step - loss: 1.3642
Epoch 3/100
125/125 - 0s - 2ms/step - loss: 1.5140
Epoch 4/100
125/125 - 0s - 2ms/step - loss: 1.8358
Epoch 5/100
125/125 - 0s - 2ms/step - loss: 1.1544
Epoch 6/100
125/125 - 0s - 2ms/step - loss: 0.9893
Epoch 7/100
125/125 - 0s - 2ms/step - loss: 1.3449
Epoch 8/100
125/125 - 0s - 2ms/step - loss: 0.8625
Epoch 9/100
125/125 - 0s - 2ms/step - loss: 1.5724
Epoch 10/100
125/125 - 0s - 2ms/step - loss: 1.7500
Epoch 11/100
125/125 - 0s - 2ms/step - loss: 1.1413
Epoch 12/100
125/125 - 0s - 2ms/step - loss: 1.1298
Epoch 13/100
125/125 - 0s - 2ms/step - loss: 2.0004
Epoch 14/100
125/125 - 0s - 2ms/step - loss: 1.0509
Epoch 15/100
125/125 - 0s - 2ms/step - loss: 0.8825
Epoch 16/100
125/125 - 0s - 2ms/step - loss: 1.1063
Epoch 17/100
125/125 - 0s - 2ms/step - loss: 0.9252
Epoch 18/100
125/125 - 0s - 2ms/step - loss: 1.1275
Epoch 19/100
125/125 - 0s - 2ms/step - loss: 1.1446
Epoch 20/100
125/125

Training your model will take longer since the Apple Quality dataset contains 4,000 items instead of only 241 in the Orange Quality dataset.

If your code was correct your should see something similiar to the following output:
~~~text
Epoch 1/100
125/125 - 1s - loss: 0.4866 - 746ms/epoch - 6ms/step
Epoch 2/100
125/125 - 0s - loss: 0.3555 - 440ms/epoch - 4ms/step
Epoch 3/100
125/125 - 0s - loss: 0.3111 - 444ms/epoch - 4ms/step
Epoch 4/100
125/125 - 0s - loss: 0.2861 - 435ms/epoch - 3ms/step
Epoch 5/100
125/125 - 0s - loss: 0.2686 - 497ms/epoch - 4ms/step

............................................

Epoch 95/100
125/125 - 0s - loss: 0.0764 - 429ms/epoch - 3ms/step
Epoch 96/100
125/125 - 0s - loss: 0.0734 - 429ms/epoch - 3ms/step
Epoch 97/100
125/125 - 0s - loss: 0.0741 - 447ms/epoch - 4ms/step
Epoch 98/100
125/125 - 1s - loss: 0.0732 - 507ms/epoch - 4ms/step
Epoch 99/100
125/125 - 0s - loss: 0.0723 - 418ms/epoch - 3ms/step
Epoch 100/100
125/125 - 0s - loss: 0.0719 - 416ms/epoch - 3ms/s
~~~

Notice that in this example, the loss went from `0.4866` after the 1st epoch down to `0.0719` after the 100th epoch.

### Example 2: Determine the Model's RSME and Accuracy

The overall objective of this assignment is to convince you that can save a _trained_ neural network to a file, and then later, recreate the neural network from the file, **_without changing the model's accuracy_**.

#### Why is this important?

As you already know, it can take significant time and processing power to train even relatively small neural networks that we created so far in this course. Neural networks that are used commercially (think "Siri" or "Alexa" or ChatGPT) are many times larger and require enormous resources as well as weeks (or months) to train. Obviously, if you had to train a neural network every time you wanted to use it, it won't be very practical and there would be little interest in "AI". However, once the neural network has been trained, you can save it to a file, and then re-use it over and over again, without any loss in the neural network's ability to solve problems (i.e. loss in accuracy).      

The code in the cell below calculates ability of the `orModel` neural network to predict an orange's quality (Y-value) based on its physical and chemical characteristics (X-values). Two measures of predictive ability are computed, the **_Root Mean Square Error (RMSE)_** and **_Accuracy_**. The code stores the RSME value in the variable `orScore` and the Accuracy value in the variable `orCorrect` and then prints out thes values.


In [5]:
# Example 2: Determine the model's RMSE and Accuracy

import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Measure RMSE error.
orPred = orModel.predict(orX)
orScore = np.sqrt(metrics.mean_squared_error(orPred,orY))
print(f"Before save score (RMSE): {orScore}")

# Measure the accuracy
orPredict_classes = np.argmax(orPred,axis=1)
orExpected_classes = np.argmax(orY,axis=1)
orCorrect = accuracy_score(orExpected_classes,orPredict_classes)
print(f"Before save Accuracy: {orCorrect}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Before save score (RMSE): 0.2628870834860242
Before save Accuracy: 0.5767634854771784


If your code is correct you should see something similar to the following output:
~~~text
8/8 [==============================] - 0s 3ms/step
Before save score (RMSE): 0.26590439677238464
Before save Accuracy: 0.5518672199170125
~~~

The `orModel` isn't doing that great ( Accuracy: 0.5518672199170125 or 55% accurate). A model with only 55% accuracy is about a accurate as flipping a coin, which is 50% accurate.

However, as mentioned above, the number of items (oranges) in this dataset is relatively small (_n_=241). As you can see, this is really too few samples for training a neural network and expect to acheive a high level of accuracy.

### **Exercise 2: Determine the Model's RSME and Accuracy**

In the cell below, determine the RSME and Accuracy of your `apModel`. Store the RSME value in a variable called `apScore` and the Accuracy value in a variable called `apCorrect`. Print out these values as shown in Example 2 above.

In [6]:
# Insert your code for Exercise 2 here

import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Measure RMSE error.
apPred = apModel.predict(apX)
apScore = np.sqrt(metrics.mean_squared_error(apPred,apY))
print(f"Before save score (RMSE): {apScore}")

# Measure the accuracy
apPredict_classes = np.argmax(apPred,axis=1)
apExpected_classes = np.argmax(apY,axis=1)
apCorrect = accuracy_score(apExpected_classes,apPredict_classes)
print(f"Before save Accuracy: {apCorrect}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Before save score (RMSE): 0.393977125450243
Before save Accuracy: 0.7795


If your code is correct you should see something similar to the following output:
~~~text
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Before save score (RMSE): 0.393977125450243
Before save Accuracy: 0.7795
~~~

According to the output above, your `apModel` is about 75% accurate when it comes to predicting apple quality. Apparently, it's a little easier to predict an apple's `Quality` with a classification neural network than to predict orange quality.

The code below sets up a neural network and reads the data (for predictions), but it does not clear the model directory or fit the neural network. The code loads the weights from the previous fit. Now we reload the network and perform another prediction. The RMSE should match the previous one exactly if we saved and reloaded the neural network correctly.

### Example 3: Save the Model

The code in the cell below saves the _trained_ neural network `orModel` as a file in two different file formats: `JSON` and `keras`.

The code saves each file in the current working directory (`save_path = "."`). The filename of the JSON file is `orModel.json` while the filename of the HDF5 file is `orModel.h5`.

In [7]:
# Example 3: Save the model

from keras.models import save_model
import os

# Save path is the current directory
save_path = "."

# Save neural network structure to JSON (no weights)
orModel_json = orModel.to_json()
with open(os.path.join(save_path,"orModel.json"), "w") as json_file:
    json_file.write(orModel_json)

# Save the model in the native Keras format
orModel.save('orModel.keras')

# Print out the files in current directory
files = os.listdir()
print(files)

['.config', 'drive', 'orModel.json', 'orModel.keras', 'sample_data']


If your code is correct you should see something like the following output:

~~~text
['.config', drive, 'orModel.json', 'orModel.keras', 'sample_data']
~~~

After running the code cell above, there should now be two new files in your `Class_02_4` folder called `orModel.jason` and `orModel.keras`.

### **Exercise 3: Save the Model**

In the code cell below save your _trained_ neural network `apModel` as a JSON file with the filename, `apModel.json`, and as a native Keras file with the filenmane `apModel.keras`. Save both files to your current working directory (`save_path = "."`).

In [8]:
# Insert your code for Exercise 3 here

from keras.models import save_model
import os

# Save path is the current directory
save_path = "."

# Save neural network structure to JSON (no weights)
apModel_json = apModel.to_json()
with open(os.path.join(save_path,"apModel.json"), "w") as json_file:
    json_file.write(apModel_json)

# Save the model in the native Keras format
apModel.save('apModel.keras')

# Print out the files in current directory
files = os.listdir()
print(files)

['.config', 'drive', 'apModel.keras', 'orModel.json', 'orModel.keras', 'apModel.json', 'sample_data']


If your code is correct you should see something similar, but not necessarily identicalm to the following output:

~~~text
['.config', drive, 'apModel.json', 'apModel.keras', 'orModel.keras', 'orModel.json', 'sample_data']
~~~

You should now see the two more files with your neural network.

The advantage of the JSON format is that it can be visually inspected -- just click on the file name in the file browser panel. The JSON file perserves the model's _architecture_ which you can see by looking at the JSON file, but if you want to use it, you will need to train all over again.

On the other hand, you can't view the contents of the HDF5 file, since it is not UTF-8 encoded (it's formated). Neverthelss, you should always save your model in the HDF5 format since this **_preserves architecture and the values of the weights_** of the model's connections. By preserving these values you don't have to waste time retraining the model again.

### Example 4: Create New Model from Saved Model

Once a trained model has been saved in the HDF5 format, it is a simple matter to read the file to make an exact copy of the model using the Keras function `load_model()` as shown in the cell below. In Example 4 we have given the re-loaded neural network the name `or2Model` to differentiate it from the one that we built previously.  

In [9]:
# Example 4: Create new model from saved model

from keras.models import load_model

# Look in current folder
save_path = "."

# Create model2 from the saved model
or2Model = load_model(os.path.join(save_path,"orModel.keras"))

# Print out model summary
or2Model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 50)                  │             450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,801 (22.66 KB)

 Trainable params: 1,933 (7.55 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,868 (15.11 KB)

If your code is correct, you should see something similar to the following output:

~~~text
Model: "sequential_1"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 50)                  │             450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │             208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 5,801 (22.66 KB)
 Trainable params: 1,933 (7.55 KB)
 Non-trainable params: 0 (0.00 B)
 Optimizer params: 3,868 (15.11 KB)
___________________________
~~~


### **Exercise 4: Create New Model from Saved Model**

In the cell below create a new neural network called `ap2Model` from the file `apModel.keras` in your current directory. Print out a summary of your new `ap2Model`.

In [10]:
# Insert your code for Exercise 4 here

from keras.models import load_model

# Look in current folder
save_path = "."

# Create model2 from the saved model
ap2Model = load_model(os.path.join(save_path,"apModel.keras"))

# Print out model summary
ap2Model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 50)                  │             450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              52 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,333 (20.84 KB)

 Trainable params: 1,777 (6.94 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,556 (13.89 KB)

If your code is correct you should see something similar to the following output:

~~~text
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 dense (Dense)               (None, 50)                400       
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 2)                 52        
                                                                 
=================================================================
Total params: 1,727
Trainable params: 1,727
Non-trainable params: 0
~~~


### Example 5: Compare the Predictive Accuracy of the Old and New Models

The code in the cell below computes the RMSE error and the Accuracy of our new model `or2Model` and compares these values with the original `orModel`. We are trying to address the question whether re-loaded model has the same accuracy as the original model?

In [11]:
# Example 5: Determine new model's RMSE and Accuracy

import numpy as np
import math

# Measure RMSE error.
or2Pred = or2Model.predict(orX)
or2Score = np.sqrt(metrics.mean_squared_error(or2Pred,orY))
print(f"Before save score (RMSE): {orScore}")
print(f"After save score (RMSE) : {or2Score}")

# Measure the accuracy
or2Predict_classes = np.argmax(or2Pred,axis=1)
or2Expected_classes = np.argmax(orY,axis=1)
or2Correct = accuracy_score(or2Expected_classes,or2Predict_classes)
print(f"Before save Accuracy: {orCorrect}")
print(f"After save Accuracy : {or2Correct}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
Before save score (RMSE): 0.2628870834860242
After save score (RMSE) : 0.2628870834860242
Before save Accuracy: 0.5767634854771784
After save Accuracy : 0.5767634854771784


If your code is correct you should see something similar to the following output:

~~~text
8/8 [==============================] - 0s 4ms/step
Before save score (RMSE): 0.26590439677238464
After save score (RMSE) : 0.26590439677238464
Before save Accuracy: 0.5518672199170125
After save Accuracy : 0.5518672199170125
~~~~

As you can see, there is **_no difference_** in the accuracy of the saved model compared to the original one. Train Once...Use Anywhere!

Big generative models like `ChatGTP` can take days or even months to train. But once they are trained and saved, they can process new data very fast.

### **Exercise 5: Compare the Predictive Accuracy of the Old and New Models**

In the cell below write the code to compute the RMSE and Accuracy values for your `ap2Model` and print out these values along with the values for your original `apModel`.

In [12]:
# Insert your code for Exercise 5 here

import numpy as np
import math

# Measure RMSE error.
ap2Pred = ap2Model.predict(apX)
ap2Score = np.sqrt(metrics.mean_squared_error(ap2Pred,apY))
print(f"Before save score (RMSE): {apScore}")
print(f"After save score (RMSE) : {ap2Score}")

# Measure the accuracy
ap2Predict_classes = np.argmax(ap2Pred,axis=1)
ap2Expected_classes = np.argmax(apY,axis=1)
ap2Correct = accuracy_score(ap2Expected_classes,ap2Predict_classes)
print(f"Before save Accuracy: {apCorrect}")
print(f"After save Accuracy : {ap2Correct}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Before save score (RMSE): 0.393977125450243
After save score (RMSE) : 0.393977125450243
Before save Accuracy: 0.7795
After save Accuracy : 0.7795


If your code is correct, you should something similar to the following output:
~~~text
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Before save score (RMSE): 0.393977125450243
After save score (RMSE) : 0.393977125450243
Before save Accuracy: 0.7795
After save Accuracy : 0.7795
~~~


Again, there is no loss in accuracy using a trained neural network that has been saved and then recreated from the saved `keras` file.

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_02_4.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

## **Lizard Tail**


## **Stable Diffusion**



![___](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Astronaut_Riding_a_Horse_%28SD3.5%29.webp/1024px-Astronaut_Riding_a_Horse_%28SD3.5%29.webp.png)

>*An image generated with Stable Diffusion 3.5 based on the text prompt*

TStable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques. The generative artificial intelligence technology is the premier product of Stability AI and is considered to be a part of the ongoing artificial intelligence boom.

It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.[3] Its development involved researchers from the CompVis Group at Ludwig Maximilian University of Munich and Runway with a computational donation from Stability and training data from non-profit organizations.[4][5][6][7]

**Stable Diffusion** is a latent diffusion model, a kind of deep generative artificial neural network. Its code and model weights have been released publicly, and it can run on most consumer hardware equipped with a modest GPU with at least 4 GB VRAM. This marked a departure from previous proprietary text-to-image models such as DALL-E and Midjourney which were accessible only via cloud services.

**Development**

Stable Diffusion originated from a project called Latent Diffusion, developed in Germany by researchers at Ludwig Maximilian University in Munich and Heidelberg University. Four of the original 5 authors (Robin Rombach, Andreas Blattmann, Patrick Esser and Dominik Lorenz) later joined Stability AI and released subsequent versions of Stable Diffusion.

The technical license for the model was released by the CompVis group at Ludwig Maximilian University of Munich. Development was led by Patrick Esser of Runway and Robin Rombach of CompVis, who were among the researchers who had earlier invented the latent diffusion model architecture used by Stable Diffusion. Stability AI also credited EleutherAI and LAION (a German nonprofit which assembled the dataset on which Stable Diffusion was trained) as supporters of the project.

**Technology**

The denoising process used by Stable Diffusion. The model generates images by iteratively denoising random noise until a configured number of steps have been reached, guided by the CLIP text encoder pretrained on concepts along with the attention mechanism, resulting in the desired image depicting a representation of the trained concept.

**Architecture**

Diffusion models, introduced in 2015, are trained with the objective of removing successive applications of Gaussian noise on training images, which can be thought of as a sequence of denoising autoencoders. The name diffusion is from the thermodynamic diffusion, since they were first developed with inspiration from thermodynamics.

Models in Stable Diffusion series before SD 3 all used a variant of diffusion models, called latent diffusion model (LDM), developed in 2021 by the CompVis (Computer Vision & Learning) group at LMU Munich.

Stable Diffusion consists of 3 parts: the variational autoencoder (VAE), U-Net, and an optional text encoder. The VAE encoder compresses the image from pixel space to a smaller dimensional latent space, capturing a more fundamental semantic meaning of the image. Gaussian noise is iteratively applied to the compressed latent representation during forward diffusion. The U-Net block, composed of a ResNet backbone, denoises the output from forward diffusion backwards to obtain a latent representation. Finally, the VAE decoder generates the final image by converting the representation back into pixel space.

The denoising step can be flexibly conditioned on a string of text, an image, or another modality. The encoded conditioning data is exposed to denoising U-Nets via a cross-attention mechanism. For conditioning on text, the fixed, pretrained CLIP ViT-L/14 text encoder is used to transform text prompts to an embedding space. Researchers point to increased computational efficiency for training and generation as an advantage of LDMs.

With 860 million parameters in the U-Net and 123 million in the text encoder, Stable Diffusion is considered relatively lightweight by 2022 standards, and unlike other diffusion models, it can run on consumer GPUs, and even CPU-only if using the OpenVINO version of Stable Diffusion.

**SD XL**
The XL version uses the same LDM architecture as previous versions, except larger: larger UNet backbone, larger cross-attention context, two text encoders instead of one, and trained on multiple aspect ratios (not just the square aspect ratio like previous versions).

The SD XL Refiner, released at the same time, has the same architecture as SD XL, but it was trained for adding fine details to preexisting images via text-conditional img2img.

**SD 3.0**

Main article: Diffusion model § Rectified flow
The 3.0 version completely changes the backbone. Not a UNet, but a Rectified Flow Transformer, which implements the rectified flow method with a Transformer.

The Transformer architecture used for SD 3.0 has three "tracks", for original text encoding, transformed text encoding, and image encoding (in latent space). The transformed text encoding and image encoding are mixed during each transformer block.

The architecture is named "multimodal diffusion transformer (MMDiT), where the "multimodal" means that it mixes text and image encodings inside its operations. This differs from previous versions of DiT, where the text encoding affects the image encoding, but not vice versa.

**Training data**

Stable Diffusion was trained on pairs of images and captions taken from LAION-5B, a publicly available dataset derived from Common Crawl data scraped from the web, where 5 billion image-text pairs were classified based on language and filtered into separate datasets by resolution, a predicted likelihood of containing a watermark, and predicted "aesthetic" score (e.g. subjective visual quality). The dataset was created by LAION, a German non-profit which receives funding from Stability AI. The Stable Diffusion model was trained on three subsets of LAION-5B: laion2B-en, laion-high-resolution, and laion-aesthetics v2 5+. A third-party analysis of the model's training data identified that out of a smaller subset of 12 million images taken from the original wider dataset used, approximately 47% of the sample size of images came from 100 different domains, with Pinterest taking up 8.5% of the subset, followed by websites such as WordPress, Blogspot, Flickr, DeviantArt and Wikimedia Commons. An investigation by Bayerischer Rundfunk showed that LAION's datasets, hosted on Hugging Face, contain large amounts of private and sensitive data.

**Training procedures**

The model was initially trained on the laion2B-en and laion-high-resolution subsets, with the last few rounds of training done on LAION-Aesthetics v2 5+, a subset of 600 million captioned images which the LAION-Aesthetics Predictor V2 predicted that humans would, on average, give a score of at least 5 out of 10 when asked to rate how much they liked them. The LAION-Aesthetics v2 5+ subset also excluded low-resolution images and images which LAION-5B-WatermarkDetection identified as carrying a watermark with greater than 80% probability. Final rounds of training additionally dropped 10% of text conditioning to improve Classifier-Free Diffusion Guidance.

The model was trained using 256 Nvidia A100 GPUs on Amazon Web Services for a total of 150,000 GPU-hours, at a cost of $600,000.

![__](https://upload.wikimedia.org/wikipedia/commons/f/f6/Stable_Diffusion_architecture.png)

>Diagram of the latent diffusion architecture used by Stable Diffusion


**Limitations**

Stable Diffusion has issues with degradation and inaccuracies in certain scenarios. Initial releases of the model were trained on a dataset that consists of 512×512 resolution images, meaning that the quality of generated images noticeably degrades when user specifications deviate from its "expected" 512×512 resolution; the version 2.0 update of the Stable Diffusion model later introduced the ability to natively generate images at 768×768 resolution. Another challenge is in generating human limbs due to poor data quality of limbs in the LAION database. The model is insufficiently trained to replicate human limbs and faces due to the lack of representative features in the database, and prompting the model to generate images of such type can confound the model. Stable Diffusion XL (SDXL) version 1.0, released in July 2023, introduced native 1024x1024 resolution and improved generation for limbs and text.

Accessibility for individual developers can also be a problem. In order to customize the model for new use cases that are not included in the dataset, such as generating anime characters ("waifu diffusion"), new data and further training are required. Fine-tuned adaptations of Stable Diffusion created through additional retraining have been used for a variety of different use-cases, from medical imaging to algorithmically generated music. However, this fine-tuning process is sensitive to the quality of new data; low resolution images or different resolutions from the original data can not only fail to learn the new task but degrade the overall performance of the model. Even when the model is additionally trained on high quality images, it is difficult for individuals to run models in consumer electronics. For example, the training process for waifu-diffusion requires a minimum 30 GB of VRAM, which exceeds the usual resource provided in such consumer GPUs as Nvidia's GeForce 30 series, which has only about 12 GB.

The creators of Stable Diffusion acknowledge the potential for algorithmic bias, as the model was primarily trained on images with English descriptions. As a result, generated images reinforce social biases and are from a western perspective, as the creators note that the model lacks data from other communities and cultures. The model gives more accurate results for prompts that are written in English in comparison to those written in other languages, with western or white cultures often being the default representation.

**End-user fine-tuning**

To address the limitations of the model's initial training, end-users may opt to implement additional training to fine-tune generation outputs to match more specific use-cases, a process also referred to as personalization. There are three methods in which user-accessible fine-tuning can be applied to a Stable Diffusion model checkpoint:

An "embedding" can be trained from a collection of user-provided images, and allows the model to generate visually similar images whenever the name of the embedding is used within a generation prompt.[44] Embeddings are based on the "textual inversion" concept developed by researchers from Tel Aviv University in 2022 with support from Nvidia, where vector representations for specific tokens used by the model's text encoder are linked to new pseudo-words. Embeddings can be used to reduce biases within the original model, or mimic visual styles.
A "hypernetwork" is a small pretrained neural network that is applied to various points within a larger neural network, and refers to the technique created by NovelAI developer Kurumuz in 2021, originally intended for text-generation transformer models. Hypernetworks steer results towards a particular direction, allowing Stable Diffusion-based models to imitate the art style of specific artists, even if the artist is not recognised by the original model; they process the image by finding key areas of importance such as hair and eyes, and then patch these areas in secondary latent space.
DreamBooth is a deep learning generation model developed by researchers from Google Research and Boston University in 2022 which can fine-tune the model to generate precise, personalised outputs that depict a specific subject, following training via a set of images which depict the subject.